# Word Embedding em Português

Use of NILC's Word Embedding  http://nilc.icmc.usp.br/embeddings to compute sentence similarity.

## Prerequisites

In [1]:
!pip install tqdm
!pip install xlrd

In [27]:
from tqdm import tqdm
import requests
import math

def downloadFile(url, filename):
    # Streaming, so we can iterate over the response.
    r = requests.get(url, stream=True)

    # Total size in bytes.
    total_size = int(r.headers.get('content-length', 0)); 
    block_size = 1024
    wrote = 0 
    with open(filename, 'wb') as f:
        for data in tqdm(r.iter_content(block_size), total=math.ceil(total_size//block_size) , unit='KB', unit_scale=True):
            wrote = wrote  + len(data)
            f.write(data)
    if total_size != 0 and wrote != total_size:
        print("ERROR, something went wrong")  

In [2]:
import zipfile
def unzipFile(path_to_zip_file, directory_to_extract_to):
    zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
    zip_ref.extractall(directory_to_extract_to)
    zip_ref.close()

In [3]:
!ls

'ls' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.


# Unzip and load data

In [4]:
urlEmb = 'http://143.107.183.175:22980/download.php?file=embeddings/glove/glove_s100.zip'
downloadFile(urlEmb, 'embedding.zip')

342kKB [01:49, 3.11kKB/s]                                                      


In [5]:
unzipFile('embedding.zip', 'embedding')
!ls embedding/

'ls' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.


In [20]:
from tqdm import tqdm
wordEmbDict = {}
with zipfile.ZipFile('embedding/glove_s50.zip', "r") as z:
    with z.open('glove_s50.txt', "r") as f: #, encoding="utf8"
    #try:
      for line in tqdm(f):
            #print(line)
            line=line.decode("utf-8")
            lineSplit = line.split(' ')
            #if len(lineSplit[0]) > 2 or lineSplit[0] in ['.',',','!',';','?',':', 'pq']:
            vec = [float(lineSplit[k+1]) for k in range(len(lineSplit)-1)]
            wordEmbDict[lineSplit[0]] = vec
    #except:
    #    o=0

934964it [00:36, 25389.72it/s]


In [7]:
import numpy as np
np.linalg.norm(wordEmbDict['?'])

7.054696011699441

# Perform cosine similarity

In [10]:
import numpy as np

def getCosineSimilarity(lstWord1, lstWord2):
    cbow1 = getContinuousBOW(lstWord1)
    cbow2 = getContinuousBOW(lstWord2)
    if cbow1 is None or cbow2 is None:
      print(lstWord1)
      return 0
    
    return np.dot(cbow1, cbow2)

def getContinuousBOW(lstWord):
    vec = None
    for w in lstWord:
        wvec = wordEmbDict.get(w, None)
        if wvec is not None:
            if vec is None:
              vec = np.array(wvec)
            else:
              vec += np.array(wvec)
    if vec is None:
        return None
    else:
        return vec / np.linalg.norm(vec)


In [11]:
getCosineSimilarity('Meu pai gosta de suco de limão'.split(' '), 'Minha mãe aprecia néctar de laranja'.split(' '))

0.7880761602559821

In [13]:
getCosineSimilarity('Meu pai gosta de suco de limão'.split(' '), 'Vamos atacar o inimigo agora'.split(' '))

0.39522174535853083

In [16]:
getCosineSimilarity('Houve um problema na bomba da refinaria'.split(' '), 'A máquina apresentou um defeito na instalação'.split(' '))

0.6656626461506673

In [22]:
getCosineSimilarity('vazamento árvore de natal'.split(' '), 'perda de hidrocarboneto'.split(' '))

0.3374617135936757

In [24]:
getCosineSimilarity('vazamento árvore de natal'.split(' '), 'visita do papai noel'.split(' '))

0.5057476093748772

# Stemming in Portuguese

In [13]:
import nltk
#nltk.download('rslp')
#nltk.download('stopwords')

In [14]:
with open('docs/politica/2018_09_politica.txt', "r") as f:
    txt = f.read()

In [15]:
nltk.corpus.stopwords.words('portuguese')

['de',
 'a',
 'o',
 'que',
 'e',
 'do',
 'da',
 'em',
 'um',
 'para',
 'com',
 'não',
 'uma',
 'os',
 'no',
 'se',
 'na',
 'por',
 'mais',
 'as',
 'dos',
 'como',
 'mas',
 'ao',
 'ele',
 'das',
 'à',
 'seu',
 'sua',
 'ou',
 'quando',
 'muito',
 'nos',
 'já',
 'eu',
 'também',
 'só',
 'pelo',
 'pela',
 'até',
 'isso',
 'ela',
 'entre',
 'depois',
 'sem',
 'mesmo',
 'aos',
 'seus',
 'quem',
 'nas',
 'me',
 'esse',
 'eles',
 'você',
 'essa',
 'num',
 'nem',
 'suas',
 'meu',
 'às',
 'minha',
 'numa',
 'pelos',
 'elas',
 'qual',
 'nós',
 'lhe',
 'deles',
 'essas',
 'esses',
 'pelas',
 'este',
 'dele',
 'tu',
 'te',
 'vocês',
 'vos',
 'lhes',
 'meus',
 'minhas',
 'teu',
 'tua',
 'teus',
 'tuas',
 'nosso',
 'nossa',
 'nossos',
 'nossas',
 'dela',
 'delas',
 'esta',
 'estes',
 'estas',
 'aquele',
 'aquela',
 'aqueles',
 'aquelas',
 'isto',
 'aquilo',
 'estou',
 'está',
 'estamos',
 'estão',
 'estive',
 'esteve',
 'estivemos',
 'estiveram',
 'estava',
 'estávamos',
 'estavam',
 'estivera',
 'es

In [17]:
docs=txt.split('*')

In [9]:
stemmer = nltk.stem.RSLPStemmer()

In [19]:
docs[0].split(' ')

['<p><strong>Alvo',
 'duplo',
 '</strong>A',
 'campanha',
 'de',
 '<a',
 'href="https://www1.folha.uol.com.br/poder/2018/09/alckmin-critica-pt-por-esconder-candidato-e-diz-que-sua-propaganda-so-reproduz-fala-de-bolsonaro.shtml">Geraldo',
 'Alckmin</a>',
 '(PSDB)',
 'vai',
 'manter',
 'Jair',
 'Bolsonaro',
 '(PSL)',
 'sob',
 '<a',
 'href="https://painel.blogfolha.uol.com.br/2018/08/30/alckmin-estreia-na-tv-com-peca-dramatica-para-comover-eleitor-de-bolsonaro-veja/">forte',
 'artilharia</a>,',
 'mas',
 'também',
 'planeja',
 'iniciar',
 'em',
 'breve',
 'ataques',
 'mais',
 'incisivos',
 'contra',
 'o',
 'PT,',
 'numa',
 'tentativa',
 'de',
 'recuperar',
 'a',
 'simpatia',
 'de',
 'eleitores',
 'antipetistas',
 'que',
 'trocaram',
 'os',
 'tucanos',
 'pelo',
 'capitão',
 'reformado',
 'do',
 'Exército.',
 'Pesquisas',
 'feitas',
 'pela',
 'equipe',
 'de',
 'Alckmin',
 'com',
 'grupos',
 'de',
 'indecisos',
 'para',
 'avaliar',
 'a',
 'repercussão',
 'dos',
 'primeiros',
 'programas',
 'd

In [25]:
import re
re.findall(r"[\w']+|[.,!?;]", docs[0])

['p',
 'strong',
 'Alvo',
 'duplo',
 'strong',
 'A',
 'campanha',
 'de',
 'a',
 'href',
 'https',
 'www1',
 '.',
 'folha',
 '.',
 'uol',
 '.',
 'com',
 '.',
 'br',
 'poder',
 '2018',
 '09',
 'alckmin',
 'critica',
 'pt',
 'por',
 'esconder',
 'candidato',
 'e',
 'diz',
 'que',
 'sua',
 'propaganda',
 'so',
 'reproduz',
 'fala',
 'de',
 'bolsonaro',
 '.',
 'shtml',
 'Geraldo',
 'Alckmin',
 'a',
 'PSDB',
 'vai',
 'manter',
 'Jair',
 'Bolsonaro',
 'PSL',
 'sob',
 'a',
 'href',
 'https',
 'painel',
 '.',
 'blogfolha',
 '.',
 'uol',
 '.',
 'com',
 '.',
 'br',
 '2018',
 '08',
 '30',
 'alckmin',
 'estreia',
 'na',
 'tv',
 'com',
 'peca',
 'dramatica',
 'para',
 'comover',
 'eleitor',
 'de',
 'bolsonaro',
 'veja',
 'forte',
 'artilharia',
 'a',
 ',',
 'mas',
 'também',
 'planeja',
 'iniciar',
 'em',
 'breve',
 'ataques',
 'mais',
 'incisivos',
 'contra',
 'o',
 'PT',
 ',',
 'numa',
 'tentativa',
 'de',
 'recuperar',
 'a',
 'simpatia',
 'de',
 'eleitores',
 'antipetistas',
 'que',
 'trocaram',


In [56]:
import pandas as pd
tagsExistentes = pd.read_excel('cpBNCC.xlsx', sheet_name='Árvore de Assuntos da Estante')
tagsExistentes.head()

,Tema,Subtema,Assunto,Assunto.1,Assunto.2,Assunto.3
0,Biologia Molecular,Bioquímica Celular,NaN,NaN,NaN,NaN
1,Biologia Molecular,Bioquímica Celular,Ácidos Nucleicos,NaN,NaN,NaN
2,Biologia Molecular,Bioquímica Celular,Enzima,NaN,NaN,NaN
3,Biologia Molecular,Bioquímica Celular,Glicídios,NaN,NaN,NaN
4,Biologia Molecular,Bioquímica Celular,Sais Minerais (Biologia),NaN,NaN,NaN


In [0]:
dfTagsMEC1 = pd.read_excel('cpBNCC.xlsx', sheet_name=2)
dfTagsMEC2 = pd.read_excel('cpBNCC.xlsx', sheet_name=3)
dfTagsMEC3 = pd.read_excel('cpBNCC.xlsx', sheet_name=4)
dfTagsMEC = pd.concat([dfTagsMEC1,dfTagsMEC2,dfTagsMEC3])

In [120]:
idQuery = 650
query = dfTagsMEC.iloc[idQuery].dropna().tolist()
query = ' '.join(query).lower().split(' ')
dfTagsMEC.iloc[idQuery]

ANO/FAIXA                                                                 7º
Ano                                                                      NaN
CAMPOS DE ATUAÇÃO                                                        NaN
COMPONENTE                                                        Matemática
EIXO                                                                     NaN
HABILIDADES                Construir triângulos, usando régua e compasso,...
Habilidade                                                               NaN
Matéria                                                                  NaN
OBJETOS DE CONHECIMENTO    Triângulos: construção, condição de existência...
PRÁTICAS DE LINGUAGEM                                                    NaN
UNIDADES TEMÁTICAS                                                 Geometria
Name: 222, dtype: object

In [121]:
cosSims = []
for index, row in tqdm(tagsExistentes.iterrows()):
    v1 = row.dropna().tolist()
    v1 = ' '.join(v1).lower().split(' ')
    cosSims.append(getCosineSimilarity(v1, query))
cosSims = np.array(cosSims)


0it [00:00, ?it/s]
106it [00:00, 1045.38it/s]
217it [00:00, 1076.70it/s]
329it [00:00, 1088.85it/s]
439it [00:00, 1090.98it/s]
550it [00:00, 1093.29it/s]
658it [00:00, 1090.19it/s]
768it [00:00, 1090.79it/s]
873it [00:00, 1084.91it/s]
893it [00:00, 1069.72it/s]

In [122]:
dfTagsMEC.iloc[idQuery]

ANO/FAIXA                                                                 7º
Ano                                                                      NaN
CAMPOS DE ATUAÇÃO                                                        NaN
COMPONENTE                                                        Matemática
EIXO                                                                     NaN
HABILIDADES                Construir triângulos, usando régua e compasso,...
Habilidade                                                               NaN
Matéria                                                                  NaN
OBJETOS DE CONHECIMENTO    Triângulos: construção, condição de existência...
PRÁTICAS DE LINGUAGEM                                                    NaN
UNIDADES TEMÁTICAS                                                 Geometria
Name: 222, dtype: object

In [123]:
np.argmax(cosSims)

508

In [124]:
tagsExistentes.iloc[cosSims.argsort()[-15:][::-1]]

,Tema,Subtema,Assunto,Assunto.1,Assunto.2,Assunto.3
508,Termologia,Estudo dos gases (Física),Lei geral dos gases perfeitos,NaN,NaN,NaN
222,Geometria Analítica,Ponto,Distância de Dois Pontos e Ponto Médio,NaN,NaN,NaN
789,Química Geral,Matéria,Estados Físicos,NaN,NaN,NaN
824,Química Geral,Polaridade das Ligações,NaN,NaN,NaN,NaN
777,Físico-Química,Soluções,Relação entre as Concentrações,NaN,NaN,NaN
440,Dinâmica,Estudo do Plano Inclinado,Componentes do Peso,NaN,NaN,NaN
654,História Geral,História Contemporânea,Independência dos EUA,NaN,NaN,NaN
439,Dinâmica,Trabalho de uma força,NaN,NaN,NaN,NaN
671,História Geral,Idade Média,Reino dos Grancos,Império Carolíngio,NaN,NaN
225,Geometria Analítica,Reta,Posições Relativas entre Retas,NaN,NaN,NaN


In [116]:
tagsExistentes.iloc[cosSims.argsort()[-15:][::-1]]

,Tema,Subtema,Assunto,Assunto.1,Assunto.2,Assunto.3
654,História Geral,História Contemporânea,Independência dos EUA,NaN,NaN,NaN
657,História Geral,História Contemporânea,Nova Ordem Econômica,NaN,NaN,NaN
439,Dinâmica,Trabalho de uma força,NaN,NaN,NaN,NaN
777,Físico-Química,Soluções,Relação entre as Concentrações,NaN,NaN,NaN
122,Evolução e Origem da vida,Seleção Natural,NaN,NaN,NaN,NaN
121,Evolução e Origem da vida,Origem da vida,NaN,NaN,NaN,NaN
119,Evolução e Origem da vida,Especiações,NaN,NaN,NaN,NaN
584,História do Brasil,Brasil Império,Segundo Reinado,Economia e Política,NaN,NaN
671,História Geral,Idade Média,Reino dos Grancos,Império Carolíngio,NaN,NaN
789,Química Geral,Matéria,Estados Físicos,NaN,NaN,NaN
